# Educarriere

In [7]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, class_name=None, style=None, text_contains=None):
    if element:
        tag = element.find(class_=class_name, style=style, text=text_contains)
        return tag.text.strip() if tag else ""
    else:
        return ""

def clean_text(text):
    if text is not None:
        cleaned_text = text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')
        return cleaned_text.strip() if cleaned_text else None
    else:
        return None

def extract_date(element, text_contains):
    date_elements = element.find_all('a', class_='text')
    date = next((e.find('span', style='color:#FF0000;').text.strip() for e in date_elements if text_contains in e.text), "")
    return date

def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=200)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='box row')

    data_list = []

    for wrapper in job_description_wrappers:
        h4_tag = wrapper.find('h4')
        poste = extract_text(h4_tag)

        entry_title_tag = wrapper.find('p', class_='entry-title')
        sous_titre = extract_text(entry_title_tag)

        a_text_tag = wrapper.find('a', class_='text')
        code = extract_text(a_text_tag, style='color:#FF0000;')

        date_edition = extract_date(wrapper, "Date d'édition:")
        date_limite = extract_date(wrapper, "Date limite:")

        pays_tag = wrapper.find('a', class_='text')
        pays = pays_tag.find_parent().text.strip().split()[-1] if pays_tag else None

        sous_titre = clean_text(sous_titre)

        data_list.append({
            'Poste': clean_text(poste),
            'Sous_titre': sous_titre,
            'Code': clean_text(code),
            'Date_DEdition': date_edition,
            'Date_limite': date_limite,
            'Pays': clean_text(pays)
        })

    df = pd.DataFrame(data_list)
    return df

# Liste des liens
urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

# Créer un DataFrame à partir des liens
result_df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Supprimer les lignes dont toutes les variables n'ont pas de données
result_df = result_df.dropna(how='any').reset_index(drop=True)

# Afficher le DataFrame
result_df

C:\Users\ngora\AppData\Local\Temp\ipykernel_13160\2078221442.py:7: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tag = element.find(class_=class_name, style=style, text=text_contains)


,Poste,Sous_titre,Code,Date_DEdition,Date_limite,Pays
0,PLUSIEURS TECHNICIENS TELECOMS (H/F),ROSAPARKSrecrutePLUSIEURS TECHNICIENS TELECOMS...,109034,23/01/2024,31/01/2024,d'I...
1,GESTIONNAIRE DE STOCK,Société Ivoirienne de Digitalisation des Resso...,109033,23/01/2024,22/02/2024,Abidjan
2,CORDISTE,AGENCE IVOIRE INTERIM (A2I)recruteCORDISTEDesc...,109032,23/01/2024,29/02/2024,ABIDJAN
3,GESTIONNAIRE DE STOCK,Société Ivoirienne de Digitalisation des Resso...,109031,23/01/2024,22/02/2024,Abidjan
4,CHEF SECTEUR BOUAFLE,Société Ivoirienne de Digitalisation des Resso...,109030,23/01/2024,22/02/2024,Bouafl�
...,...,...,...,...,...,...
516,STAGE DE SOUTENANCE (RH-COM; GESCOM; FCGE; RIT...,Description du posteStage de soutenanceNous of...,95492,14/02/2023,27/03/2024,...
517,ESTHÉTICIENNE - PROTHÉSISTE ONGULAIRE (H/F),Description du posteNous recherchons des esthé...,94035,16/01/2023,23/04/2025,...
518,CONSULTANT FORMATEUR EN DROIT,"Description du posteNous, cabinet de formation...",86696,27/07/2022,02/03/2024,ABIDJAN
519,CONSULTANT FORMATEUR SPECIALISE,"Description du posteNous, cabinet de formation...",86695,27/07/2022,02/03/2024,ABIDJAN


In [2]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens
urls = ["https://emploi.educarriere.ci/nos-offres?page1={}&codes=&mots_cles=&typeemploi1=&niveau1=&anciennete=&typeoffre1=&recruteur=".format(category) for category in range(40)]

# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens
for url in urls:
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='box row')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <h4> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('h4')

        # Vérifier si la balise <h4> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag.find('a')['href']
            all_job_details.append({'Offre_Link': offre_link})

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les détails des emplois en un seul DataFrame
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
    # Afficher le DataFrame
    #print(all_job_details_df)
else:
    print("Aucun détail d'offre d'emploi trouvé.")


In [8]:
all_job_details_df

,Offre_Link
0,https://emploi.educarriere.ci/offre-113198-com...
1,https://emploi.educarriere.ci/offre-113197-log...
2,https://emploi.educarriere.ci/offre-113196-une...
3,https://emploi.educarriere.ci/offre-113195-une...
4,https://emploi.educarriere.ci/offre-113194-un-...
...,...
507,https://emploi.educarriere.ci/offre-99669-stag...
508,https://emploi.educarriere.ci/offre-98212-esth...
509,https://emploi.educarriere.ci/offre-90873-cons...
510,https://emploi.educarriere.ci/offre-90872-cons...


In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3
import time

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# (votre fonction extract_job_information reste inchangée)
def extract_job_information(soup):
    # Extraction des informations de l'offre d'emploi
    poste = soup.select_one('li.list-group-item:-soup-contains("Poste")').strong.next_sibling.strip()
    type_offre = soup.select_one('li.list-group-item:-soup-contains("Type d\'offre")').strong.next_sibling.strip()
    metiers = soup.select_one('li.list-group-item:-soup-contains("Métier(s):")').strong.next_sibling.strip()
    niveaux = soup.select_one('li.list-group-item:-soup-contains("Niveau(x):")').strong.next_sibling.strip()
    experience = soup.select_one('li.list-group-item:-soup-contains("Expérience:")').strong.next_sibling.strip()
    lieu = soup.select_one('li.list-group-item:-soup-contains("Lieu:")').strong.next_sibling.strip()
    
    # Extraction des dates de publication et de limite
    date_publication = soup.find('strong', string='Date de publication:').find_next('span').text.strip()
    date_limite = soup.find('strong', string='Date limite:').find_next('span').text.strip()
    
   # description = soup.select_one('div.text-col.post.small-post.col-md-9.col-xs-12 ul.list-group').text.strip()
    description = soup.select_one('div.entry-content').text.strip()

    return {
        "Poste": [poste],
        "Type d'offre": [type_offre],
        "Métier(s)": [metiers],
        "Niveau(x)": [niveaux],
        "Expérience": [experience],
        "Lieu": [lieu],
        "Date de publication": [date_publication],
        "Date limite": [date_limite],
        "Description": [description]
    }

# Liste des URLs à scraper
urls = list(all_job_details_df['Offre_Link'])

# Liste pour stocker les DataFrames
dfs = []

# Boucle sur chaque URL
for url in urls:
    try:
        # Envoyer une requête GET au site avec un délai de 120 secondes
        response = requests.get(url, headers=headers, verify=True, timeout=120)

        # Vérifier si la requête a réussi (statut 200)
        if response.status_code == 200:
            # Analyser le contenu de la page avec BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            try:
                # Extraire les informations sur l'emploi
                job_info = extract_job_information(soup)

                # Créer un DataFrame
                df = pd.DataFrame(job_info)

                # Ajouter le DataFrame à la liste
                dfs.append(df)
            except Exception as e:
                print(f"An error occurred while extracting job information: {e}")
        else:
            print(f"Échec de la requête pour l'URL {url}. Statut : {response.status_code}")

    except requests.exceptions.Timeout:
        print(f"Timeout lors de la requête pour l'URL {url}")
    except requests.exceptions.RequestException as e:
        print(f"Une erreur s'est produite lors de la requête pour l'URL {url}: {e}")

    # Ajouter un délai de 5 secondes entre les requêtes pour éviter d'être bloqué
    time.sleep(5)

# Concaténer tous les DataFrames en un seul DataFrame
df_Educarriere = pd.concat(dfs, ignore_index=True)
df_Educarriere


Une erreur s'est produite lors de la requête pour l'URL https://emploi.educarriere.ci/offre-113195-une-01-stagiaire-assistante-de-direction.html: HTTPSConnectionPool(host='emploi.educarriere.ci', port=443): Read timed out.
Une erreur s'est produite lors de la requête pour l'URL https://emploi.educarriere.ci/offre-113193-annonce-charge-de-formation.html: HTTPSConnectionPool(host='emploi.educarriere.ci', port=443): Read timed out.
Timeout lors de la requête pour l'URL https://emploi.educarriere.ci/offre-113189-responsable-garage.html
Une erreur s'est produite lors de la requête pour l'URL https://emploi.educarriere.ci/offre-113171-recrutement-de-stagiaires.html: ('Connection broken: IncompleteRead(16202 bytes read, 4328 more expected)', IncompleteRead(16202 bytes read, 4328 more expected))
Une erreur s'est produite lors de la requête pour l'URL https://emploi.educarriere.ci/offre-113148-commerciaux.html: HTTPSConnectionPool(host='emploi.educarriere.ci', port=443): Read timed out.
Une err

,Poste,Type d'offre,Métier(s),Niveau(x),Expérience,Lieu,Date de publication,Date limite,Description
0,COMMUNITY MANAGER,Emploi,"Communication, Infographie, Marketing, Télécom...","BAC+2, BAC+3, BAC+4",2 ans,Riviera triangle,22/01/2024,04/02/2024,FANGAN TECHrecruteCOMMUNITY MANAGER\n \n \nDes...
1,LOGISTICIEN DE CHANTIER,Emploi,"Génie Civil/Travaux publics, Logistique/Transport","BAC+2, BAC+3, BAC+4",5 ans,ABIDJAN,22/01/2024,28/01/2024,PRESTICOMrecruteLOGISTICIEN DE CHANTIER\n \n \...
2,UNE (01) STAGIAIRE ASSISTANTE DE DIRECTION,Stage,Assistanat de Direction,"BAC+2, BAC+1",2 ans,Côte d'Ivoire,22/01/2024,22/01/2024 (Expirée),Description du posteMD Holding International\n...
3,UN (01) STAGIAIRE CHARGE DAPPEL DOFFRES,Stage,"Commerce et Administration des Entreprises, Co...","BAC+4, BAC+5",3 ans,Côte d'Ivoire,22/01/2024,05/02/2024,MDHoldingrecruteUN (01) STAGIAIRE CHARGE D’APP...
4,ASSISTANT QUALITE,Emploi,Qualité,BAC+3,5 ans,Abidjan,22/01/2024,05/02/2024,PIPELINE CONSTRUCTIONrecruteASSISTANT QUALITE\...
...,...,...,...,...,...,...,...,...,...
477,STAGE DE SOUTENANCE (RH-COM; GESCOM; FCGE; RIT...,Stage,"Communication, Commerce/Ventes, Finances/Compt...",BAC+2,,"Abidjan, Cocody blockhaus",14/02/2023,27/03/2024,Description du posteStage de soutenance\r\nNou...
478,ESTHÉTICIENNE - PROTHÉSISTE ONGULAIRE (H/F),Stage,Esthétique/Beauté,BAC,,Angré 8e tranche,16/01/2023,23/04/2025,Description du posteNous recherchons des esthé...
479,CONSULTANT FORMATEUR EN DROIT,Consultance,"Fiscalité, Juridique/Droit",BAC+5,10 ans,ABIDJAN,27/07/2022,02/03/2024,"Description du poste\nNous, cabinet de formati..."
480,CONSULTANT FORMATEUR SPECIALISE,Consultance,"Agriculture, Aménagement du Territoire , Archi...",BAC+4,10 ans,ABIDJAN,27/07/2022,02/03/2024,"Description du poste\nNous, cabinet de formati..."


In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_Educarriere['Poste'] = list(result_df['Poste'])
df_Educarriere['Sous_titre'] = list(result_df['Sous_titre'])
df_Educarriere['Code'] = list(result_df['Code'])
df_Educarriere['Date_DEdition'] = list(result_df['Date_DEdition'])
df_Educarriere['Date_limite'] = list(result_df['Date_limite'])
df_Educarriere['Pays'] = list(result_df['Pays'])



# Réorganiser les colonnes selon vos besoins
df_Educarriere 

# NOVOJOB

In [10]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie
categories = [
    "toutes les offres d'emploi",
    "juridique,fiscal,audit,conseil",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "metiers banque et assurances",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",
    "Métiers de l'agriculture"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

intitules_list = []
entreprises_list = []
pays_list = []
dates_list = []
lien_list = []
niveau_list = []
experience_list = []

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    if 'finance' in category_link:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')
    else:
        offres = soup.find_all('h2', class_='ellipsis row-fluid')
        entreprises = soup.find_all('h6', class_='ellipsis')
        niveaux = soup.find_all('span', class_='spaced-right phone-display-blok')

    for offre, entreprise, niveau in zip(offres, entreprises, niveaux):
        bloc_bottom = offre.find_next('div', class_='bloc-bottom')
        intitules_list.append(offre.get_text().strip())
        entreprises_list.append(entreprise.get_text().strip())
        lien_list.append(category_links.index(category_link))

        # Les informations (pays, date, niveau, expérience) sont contenues dans la même span, nous devons les séparer
        pays_info = bloc_bottom.find('i', class_='fa fa-map-marker icon-left')
        pays = pays_info.find_parent().text.strip() if pays_info else None
        pays_list.append(pays)

        date_info = bloc_bottom.find('i', class_='fa fa-clock-o icon-left')
        date = date_info.find_parent().text.strip() if date_info else None
        dates_list.append(date)

        # Ajout des colonnes pour le niveau du poste et l'expérience demandée
        niveau_info = niveau.find('i', class_='fa fa-bookmark icon-left')
        niveau_text = niveau_info.find_parent().text.strip() if niveau_info else None

        # Utiliser une expression régulière pour extraire les informations de niveau et d'expérience
        match = re.match(r'(.+) \((.+)\)', niveau_text)

        if match:
            niveau_col, experience_col = match.groups()
        else:
            niveau_col, experience_col = None, None

        niveau_list.append(niveau_col)
        experience_list.append(experience_col)



In [11]:
df_offers = pd.DataFrame({
    'Intitule': intitules_list,
    'Entreprise': entreprises_list,
    'Pays': pays_list,
    'Date': dates_list,
    'Niveau': niveau_list,
    'Experience_lettre': experience_list,
    'Lien': lien_list
})
df_offers

,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien
0,Livreurs Moto,Entreprise anonyme,Côte d'ivoire,04 Janvier,Confirmé / Expérimenté,Sans expérience,0
1,Pompiste H/F,Coris consulting,"Abidjan, Côte d'ivoire",27 Décembre 2023,Débutant / Junior,Moins d’un an,0
2,Caissier H/F,Coris consulting,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,0
3,Ouvrier de rayon,BICICI,Côte d'ivoire,27 Décembre 2023,Débutant / Junior,Sans expérience,0
4,Directeur d'Agence (h/f),Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",23 Novembre 2023,Confirmé / Expérimenté,3 à 5 ans,0
...,...,...,...,...,...,...,...
621,Stagiaire Gestion de Projets Digitaux,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",29 Novembre 2023,Débutant / Junior,Moins d’un an,25
622,Auditeur Senior des Systèmes d'Information,Société Générale Afrique de l'ouest ( Centre ...,"Abidjan, Côte d'ivoire",09 Novembre 2023,Responsable d'équipe,3 à 5 ans,25
623,Technicien Génie Civil Option Bâtiment,SOCOPI,Côte d'ivoire,17 Janvier,Débutant / Junior,1 à 2 ans,25
624,Chargé de Support Système Windows et Virtualis...,Ascens,"Abidjan, Côte d'ivoire",24 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,25


In [12]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens pour chaque catégorie
categories = [
    "toutes les offres d'emploi",
    "juridique,fiscal,audit,conseil",
    "administrations,moyens généraux",
    "assistanat,secrétariat",
    "metiers banque et assurances",
    "RH,personnel,formation",
    "education,enseignement",
    "direction générale,direction d'unité",
    "vente,televente,assistanat",
    "commercial,technico commercial,service client",
    "responsable commercial,grands comptes",
    "créatio, design",
    "marketing, communication",
    "journalisme,médias,traduction",
    "informatique,systèmes d'information,internet",
    "télécommunication,réseaux",
    "chantier,métiers BTP,architecture",
    "ingénierie,etudes,projet,R&D",
    "logistique,achat,stock,transport",
    "production,méthode,industrie",
    "maintenance,entretien",
    "Qualité,sécurité,Environnement",
    "Santé,Médical,Pharmacie",
    "Hotellerie,Tourisme,Restauration, Loisirs",
    "Ouvriers qualifiés, Chauffeur",
    "autre",
    "Métiers de l'agriculture"
]

base_url = "https://www.novojob.com/cote-d-ivoire/offres-d-emploi?q="
category_links = [f"{base_url}{'+'.join(category.split(','))}" for category in categories]

# Utilisation d'un en-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

# Initialiser le pilote Selenium
driver = webdriver.Chrome()

# Liste pour stocker les détails de chaque emploi
all_job_lien = []

# Parcourir les liens de chaque catégorie
for category_link in category_links:
    req = requests.get(category_link, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_='row-fluid job-details pointer')

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <a> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('a')

        # Vérifier si la balise <a> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag['href']

            
            # Ajouter les détails de l'emploi à la liste
            all_job_lien.append(offre_link)

# Fermer le pilote Selenium à la fin
driver.quit()

# Convertir les détails des offres d'emploi en DataFrame
# Convertir les détails des offres d'emploi en DataFrame
df_Novojob = pd.DataFrame(all_job_lien, columns=['Offre_Link'])
df_Novojob.head()


,Offre_Link
0,https://www.novojob.com/cote-d-ivoire/offres-d...
1,https://www.novojob.com/cote-d-ivoire/offres-d...
2,https://www.novojob.com/cote-d-ivoire/offres-d...
3,https://www.novojob.com/cote-d-ivoire/offres-d...
4,https://www.novojob.com/cote-d-ivoire/offres-d...


In [ ]:

import requests
import pandas as pd

# Example usage for multiple job URLs
job_urls = list(df_Novojob['Offre_Link'])

# Extract details for each job URL
all_job_details = []
for url in job_urls:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')

    job_details = {}

    # Extracting job details
    details_section = soup.find('ul', class_='text-small')
    if details_section:
        for li in details_section.find_all('li', class_='row-fluid'):
            key = li.find('span', class_='span4').text.strip()
            value = li.find('span', class_='span8').text.strip()
            job_details[key] = value

    # Extracting the provided text
    description_section = soup.find('div', class_='spaced details-description')
    if description_section:
        provided_text = description_section.text.strip()
        job_details['Provided Text'] = provided_text

    all_job_details.append(job_details)

# Create a DataFrame
df_offers = pd.DataFrame(all_job_details)
df_offers

In [ ]:
# Ajouter les listes existantes en tant que colonnes au DataFrame
df_offers['Intitule'] = intitules_list
df_offers['Entreprise'] = entreprises_list
df_offers['Pays'] = pays_list
df_offers['Date'] = dates_list
df_offers['Niveau'] = niveau_list
df_offers['Experience_lettre'] = experience_list
df_offers['Lien'] = lien_list

# Réorganiser les colonnes selon vos besoins

df_offers

In [25]:

# Réorganiser les colonnes selon vos besoins
df_offers = df_offers[[
    'Intitule', 'Entreprise', 'Pays', 'Date', 'Niveau', 'Experience_lettre',
    'Lien', "Lieu de travail", "Date d'expiration", 'Niveau de poste', "Secteur d'activité", "Niveau d'étude (diplome)",
    "Nombre de postes", "Type de contrat", "Provided Text", "Nom de l'entreprise"
]]


df_offers


,Intitule,Entreprise,Pays,Date,Niveau,Experience_lettre,Lien,Lieu de travail,Date d'expiration,Niveau de poste,Secteur d'activité,Niveau d'étude (diplome),Nombre de postes,Type de contrat,Provided Text,Nom de l'entreprise
0,Consultant Manager Général,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Manager / Responsable département,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Manager / Responsable département,Services,"Master 1, Licence Bac + 4| Master 2, Ingénior...",01,Consultant,Notre client une société qui vient de se const...,NaN
1,Administrateur Systèmes et Réseaux,Société Ivoirienne de Banque (SIB),"Abidjan, Côte d'ivoire",03 Janvier,Confirmé / Expérimenté,3 à 5 ans,0,"Abidjan, Côte d'ivoire",15 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",02,CDI,Missions du posteSous la supervision du Respon...,NaN
2,HRBP Senior – H/F,Société Générale Côte D'ivoire,Côte d'ivoire,30 Novembre 2023,Confirmé / Expérimenté,6 à 10 ans,0,Côte d'ivoire,28 Février,Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
3,Directeur d'Agence (h/f),BICICI,"Abidjan, Côte d'ivoire",23 Novembre 2023,Confirmé / Expérimenté,3 à 5 ans,0,"Abidjan, Côte d'ivoire",21 Février,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 2, Ingéniorat, Bac + 5",01,CDI,AVIS DE RECRUTEMENT FAMILLE D'EMPLOIS : DEVELO...,NaN
4,Chargé de Prévention et Sécurité au Travail (...,Entreprise anonyme,"Abidjan, Côte d'ivoire",23 Octobre 2023,Confirmé / Expérimenté,3 à 5 ans,0,"Abidjan, Côte d'ivoire",21 Janvier,Confirmé / Expérimenté,"Banque, Assurance, Finance","Master 1, Licence Bac + 4| Master 2, Ingénior...",01,CDI| CDD,La banque en toute confiance. Acteur majeur du...,NaN
5,Gestionnaire Tech Bar,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le TECH BAR est un coworking qui permettra aux...,NaN
6,Gestionnaire Centre Financier,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Master 1, Licence Bac + 4",01,CDI| CDD,Le CENTRE FINANCIER est une unité au sein de l...,NaN
7,Gestionnaire Food Court,Exceliam,"Abidjan, Côte d'ivoire",06 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,"Abidjan, Côte d'ivoire",04 Février,Responsable d'équipe| Confirmé / Expérimenté,Services,"Licence (LMD), Bac + 3| Master 1, Licence Bac...",01,CDI| CDD,Le FOOD COURT est un restaurant qui permettra ...,NaN
8,Superviseur Maintenance Bâtiments - H/F,Société Générale Côte D'ivoire,Côte d'ivoire,30 Novembre 2023,Responsable d'équipe,6 à 10 ans,0,Côte d'ivoire,28 Février,Responsable d'équipe| Confirmé / Expérimenté,Industries,"Master 2, Ingéniorat, Bac + 5",01 poste ouvert,CDI| CDD,ENTITE : ...,Société Générale Côte D'ivoire
9,Chef de Projet ( Gestion Applicative & Projets...,Société Générale Côte D'ivoire,Côte d'ivoire,30 Octobre 2023,Confirmé / Expérimenté,6 à 10 ans,0,Côte d'ivoire,28 Janvier,Confirmé / Expérimenté,Industries,"Master 1, Licence Bac + 4| Master 2, Ingénior...",02 postes ouverts,CDD,ENTITE : Sociét...,Société Générale Côte D'ivoire


# Emploi.ci

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    tag = element.find(tag_name)
    return tag.text.strip() if tag else ""

def clean_text(text):
    return text.replace('D\x92', ' ').replace('d\x92', ' ').replace('\x92', ' ').replace('\r\n', '').replace('\xa0', '')

def scrape_emploi_ci(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_description_wrappers = soup.find_all('div', class_='job-description-wrapper')

    data_list = []

    for wrapper in job_description_wrappers:
        h5_tag = wrapper.find('h5')
        poste = extract_text(h5_tag, 'a')

        job_recruiter_tag = wrapper.find('p', class_='job-recruiter')
        date_and_company = job_recruiter_tag.text.strip().split('|')
        date = date_and_company[0].strip() if date_and_company else ""
        entreprise = extract_text(job_recruiter_tag, 'a')

        description_tag = wrapper.find('div', class_='search-description')
        description = clean_text(description_tag.text.strip()) if description_tag else ""

        region_tag = wrapper.find('p', text='Région de :')
        region = extract_text(region_tag) if region_tag else ""

        data_list.append({
            'Poste': poste,
            'Entreprise': entreprise,
            'Date': date,
            'Description': description,
            'Région': region
        })

    df = pd.DataFrame(data_list)
    return df

# Liste des liens
categories = ["31", "1127", "29", "37", "1115", "30", "1115", "32", "33", "34", "35", "36", "37", "39", "38", "40", "525", "41", "28"]
#categories=["31"]
# Liste d'URLs générées
urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]

# Créer un DataFrame à partir des liens
df = pd.concat([scrape_emploi_ci(url) for url in urls], ignore_index=True)

# Afficher le DataFrame
df



In [ ]:
import time
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

# Liste des liens
urls = ["https://www.emploi.ci/recherche-jobs-cote-ivoire/?f%5B0%5D=im_field_offre_metiers%3A{}".format(category) for category in categories]


# En-tête pour éviter d'être bloqué
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Pour exécuter le navigateur en arrière-plan
options.add_argument("--disable-gpu")  # Désactiver l'accélération GPU en mode headless
chrome_driver_path = "C:\\Users\\ngora\\OneDrive\\Bureau\\INS_DATA\\chromedriver_win32\\chromedriver.exe"
options.binary_location = "C:\\Program Files\\Google\\Chrome\\Application\\chrome.exe"  # Remplacez par l'emplacement réel de votre Chrome binary
options.add_argument(f"webdriver.chrome.driver={chrome_driver_path}")
driver = webdriver.Chrome(options=options)

# Liste pour stocker les détails de chaque emploi
all_job_details = []

# Parcourir les liens
for url in urls:
    req = requests.get(url, headers=headers)
    soup = BeautifulSoup(req.text, 'html.parser')
    time.sleep(5)  # Attendre 5 secondes avant la prochaine requête

    offres = soup.find_all('div', class_="job-description-wrapper")

    # Parcourir les offres d'emploi sur la page principale
    for offre in offres:
        # Trouver la balise <h4> dans la structure HTML pour extraire le lien
        offre_link_tag = offre.find('h5')

        # Vérifier si la balise <h4> a été trouvée
        if offre_link_tag:
            # Extraire le lien de l'attribut 'href'
            offre_link = offre_link_tag.find('a')['href']
            all_job_details.append({'Offre_Link': "https://www.emploi.ci"+offre_link})

# Fermer le pilote Selenium à la fin
driver.quit()

# Concaténer tous les détails des emplois en un seul DataFrame
if all_job_details:
    all_job_details_df = pd.DataFrame(all_job_details)
    # Afficher le DataFrame
    #print(all_job_details_df)
else:
    print("Aucun détail d'offre d'emploi trouvé.")


In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from requests.exceptions import ChunkedEncodingError, ConnectionError, ReadTimeout

# Fonction pour extraire les informations d'une page
def extract_information(url):
    try:
        response = requests.get(url, timeout=120)  # Augmentation du délai à 20 secondes
        response.raise_for_status()
        response.encoding = 'utf-8'

        soup = BeautifulSoup(response.content, 'html.parser')

        # Extraction des informations sur l'entreprise
        company_info = soup.select_one('.job-ad-company')
        entreprise = {
            'Nom': company_info.select_one('.company-title a').text.strip() if company_info and company_info.select_one('.company-title a') else None,
            'Secteur d´activité': ', '.join(item.text.strip() for item in company_info.select('.sector-title .field-item')) if company_info and company_info.select('.sector-title .field-item') else None,
            'Description de l\'entreprise': soup.select_one('.job-ad-company-description label + *').text.strip() if soup.select_one('.job-ad-company-description label + *') else None
        }

        # Extraction des informations sur l'annonce
        annonce_info = soup.select_one('.job-ad-details')
        annonce = {
            'Poste': soup.select_one('.ad-ss-title').text.strip() if soup.select_one('.ad-ss-title') else None,
            'Missions': [li.text.strip() for li in soup.select('.content ul.missions li')] if soup.select('.content ul.missions') else None,
            'Profil recherché': [li.text.strip() for li in soup.select('.content ul.profil li')] if soup.select('.content ul.profil') else None,
            'Métier': soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-metiers .field-item') else None,
            'Secteur d´activité (de l\'annonce)': soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-secteur .field-item') else None,
            'Type de contrat': soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-contrat-type .field-item') else None,
            'Région': soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-region .field-item') else None,
            'Ville': soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-ville .field-item') else None,
            'Niveau d\'expérience': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-experience .field-item') else None,
            'Niveau d\'études': soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item').text.strip() if soup.select_one('.job-ad-criteria .field-name-field-offre-niveau-etude .field-item') else None,
            'Compétences clés': [li.text.strip() for li in soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item')] if soup.select('.job-ad-criteria .field-name-field-offre-tags .field-item') else None,
            'Nombre de poste(s)': soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td').text.strip() if soup.select_one('.job-ad-criteria td:contains("Nombre de poste(s) :") + td') else None,
}

        return {'entreprise': entreprise, 'annonce': annonce}

    except (ConnectionError, ReadTimeout, ChunkedEncodingError) as e:
        print(f"Erreur lors de la requête {url}: {e}")
        # Relancer la requête
        return None

# Liste des URLs
urls = list(all_job_details_df['Offre_Link'])

# Initialisation d'une liste pour stocker les DataFrames
df_list = []

# Boucle à travers chaque URL
for url in urls:
    data = extract_information(url)

    # Si la requête a échoué, passez à l'URL suivante
    if data is None:
        continue

    # Création du DataFrame pour chaque URL
    df = pd.DataFrame([data['entreprise'] | data['annonce']])

    # Ajout du DataFrame à la liste
    df_list.append(df)

# Concaténation des DataFrames de chaque URL
result_df = pd.concat(df_list, ignore_index=True)
result_df


# ci.talent

In [ ]:

import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    tag = element.find(tag_name)
    return tag.text.strip() if tag else ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_talent_com(url):
    try:
        response = requests.get(url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {url} : {e}")
        return pd.DataFrame()

    soup = BeautifulSoup(response.text, 'html.parser')

    job_wrappers = soup.find_all('div', class_='card card__job')

    data_list = []

    for wrapper in job_wrappers:
        title_tag = wrapper.find('h2', class_='card__job-title')
        title = extract_text(title_tag, 'a')
        
        employer_location_tag = wrapper.find('div', class_='card__job-empnameLocation')
        #employer = extract_text(employer_location_tag.find('div', class_='card__job-location'))  # Extract location from the inner div
        location= extract_text(employer_location_tag, 'div')

        employer_location_tag = wrapper.find('div', class_='card__job-empname-label')
        
        # Extracting employer and description from the div
        employer = employer_location_tag.text.strip() if employer_location_tag else None  # Extract location from the inner div
        
        description_tag = wrapper.find('div', class_='card__job-snippet-logo')
        description = clean_text(extract_text(description_tag, 'p'))

        data_list.append({
            'Title': title,
            'Location': location,
            'Employer': employer,
            'Description': description
        })

    df = pd.DataFrame(data_list)
    return df

# List of URLs for talent.com jobs
urls = [
    "https://ci.talent.com/jobs",
    # Add more URLs as needed
]

# Initialize an empty DataFrame to store the results
combined_df = pd.DataFrame()

# Scrape job information for each URL and concatenate the results
for url in urls:
    df = scrape_talent_com(url)
    combined_df = pd.concat([combined_df, df], ignore_index=True)

# Display the combined DataFrame
combined_df


# projobivoire

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def extract_text(element, tag_name=None):
    if element and tag_name:
        tag = element.find(tag_name)
        return tag.text.strip() if tag else ""
    return ""

def clean_text(text):
    return text.replace('\r\n', '').replace('\xa0', '')

def scrape_projobivoire_page(page_url):
    try:
        response = requests.get(page_url, timeout=500)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion à {page_url} : {e}")
        return None

    soup = BeautifulSoup(response.text, 'html.parser')

    job_items = soup.find_all('div', class_='loop-item-wrap list')

    if not job_items:
        print("Aucun élément de travail trouvé.")
        return None

    job_data_list = []

    for job_item in job_items:
        title_tag = job_item.find('h3', class_='loop-item-title')
        title = extract_text(title_tag, 'a')

        job_type_tag = job_item.find('span', class_='job-type')
        job_type = extract_text(job_type_tag, 'span')

        job_date_posted = soup.find('span', class_='job-date__posted').text.strip()
        job_date_closing = soup.find('span', class_='job-date__closing').text.strip()
        job_date_closing = job_date_closing.lstrip('-').strip()

        category_tag = job_item.find('span', class_='job-category')
        category = extract_text(category_tag, 'a')

        email_url_tag = job_item.find('span', class_='noo-tool-email-job')
        email_url = email_url_tag['data-url'] if email_url_tag else ""

        data = {
            'Title': title,
            'Type': job_type,
            'DatePosted': job_date_posted,
            'DateClosing': job_date_closing,
            'Category': category,
            'EmailURL': email_url,
        }

        job_data_list.append(data)

    return job_data_list

def scrape_multiple_pages(page_urls):
    all_job_data = []

    for page_url in page_urls:
        job_data_list = scrape_projobivoire_page(page_url)
        if job_data_list:
            all_job_data.extend(job_data_list)

    return pd.DataFrame(all_job_data)

# Liste d'URLs de pages avec plusieurs offres d'emploi
#page_urls = ["https://projobivoire.com/page/2/", "https://projobivoire.com/page/3/"]
page_urls = ["https://projobivoire.com/page/{}/".format(category) for category in range(546)]


# Scraping des détails de chaque offre d'emploi sur toutes les pages
df = scrape_multiple_pages(page_urls)

# Affichage du DataFrame
df
